In [1]:
version = "2.1.2"

<a id='toc'></a>  
# Table of Contents
- **[Assignment Description](#Topic0)**
- **[Topic 1 - Text Processing](#Topic1)**
  - [Task 1](#t1)
- **[Topic 2 - Latent Semantic Indexing](#Topic2)**
  - [Task 2a](#t2a)
  - [Task 2b](#t2b)
- **[Topic 3 - Semantic Similarity](#Topic3)**
  - [Task 3](#t3_)
  - [Task 3a](#t3a)
  - [Task 3b](#t3b)
  - [Task 3c](#t3c) 
- **[Topic 4 - Topic Coherence](#Topic4)**
  - [Task 4a](#t4a)
  - [Task 4b](#t4b)

In [2]:
# Either of the following is no longer
# necessary for matplotlib in notebooks.
# The import statement below has you covered!

# %matplotlib notebook
# %matplotlib inline

<a id='Topic0'></a>
# SIADS 543 Assignment 3:
## Text representations, topic modeling and word embeddings

In this week's assignment you'll gain experience applying topic modeling and other latent variable estimation methods. We'll focus on textual data, continuing to work with vectorizers and related text representations like embeddings.

All questions in this assignment are auto-graded. Some parts ask you a short question or two about on the results: these are meant to encourage you to reflect on the outcomes, but do not need to be included as part of your graded submission.  

<a href='#toc'>TOC</a>

In [3]:
# Suppress all warnings only when absolutely necessary
# Warnings are in place for a reason!
import warnings

# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

In [4]:
# First import some necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

up, down = True, False
pd.set_option('display.max_rows', 25)

In [5]:
np.set_printoptions(precision=3)

## Additional imports can be inlcuded here

### The following cell contains a couple of helpful utility functions for use with this assignment.

In [6]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# display_topics:  example showing how to take the model components generated by LDA or NMF
# and use them to dump the top words by weight for each topic.


def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(
            " ".join(
                [feature_names[i] for i in topic.argsort()[: -num_top_words - 1 : -1]]
            )
        )


# load_newsgroup_documents: prepare training and test data from the 20newsgroups dataset
def load_newsgroup_documents():
    # The Coursera environment must be self-contained and so APIs that do external fetching
    # aren't allowed. So we use pickle files that can be stored locally instead of the following
    # API calls.
    # dataset_train   = fetch_20newsgroups(subset = 'train', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
    # dataset_test    = fetch_20newsgroups(subset = 'test', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

    pickle_train_data = open("../../voc/public/assets/20newsgroups_train_data.pickle", "rb")
    pickle_train_labels = open("../../voc/public/assets/20newsgroups_train_labels.pickle", "rb")
    documents_train = pickle.load(pickle_train_data)
    labels_train = pickle.load(pickle_train_labels)
    pickle_train_data.close()
    pickle_train_labels.close()

    return documents_train, labels_train


# load the dataset for future use....
documents_train, labels_train = load_newsgroup_documents()

<a id='Topic1'></a>
## Topic 1 - Text Processing (20 points).
### The choice of text processing can impact final classification performance.

There are many different parameter settings for Vectorizer objects in scikit-learn. Small changes in these settings can result in very different text representations and significant changes in final classifier accuracy. For this Task you'll train a commonly-used type of text classifier, Multinomial Naive Bayes, using three different input representations for text, to see the effect of different parameter choices on classifier training set accuracy.  

<a href='#toc'>TOC</a>

<a id='t1'></a>
## Task 1 Create and train multiple vectorizers (20 points).
Follow these steps:
1. Create a TfidfVectorizer object (let's call it A) with the following settings:

    `max_features = 10000, # only top 10k by freq`
    
    `lowercase = False, # keep capitalization`
    
    `ngram_range = (1,2), # include 2-word phrases`
    
    `min_df=10,  # note: absolute count of documents`
    
    `max_df=0.95,   # note: % of docs in collection`
    
    `stop_words='english'`
    
    
2. Create a CountVectorizer object (let's call it B) with the same settings:

    `max_features = 10000, # only top 10k by freq`
    
    `lowercase = False, # keep capitalization`
    
    `ngram_range = (1,2), # include 2-word phrases`
    
    `min_df=10,  # note: absolute count of doc`
    
    `max_df=0.95,   # note: % of docs`
    
    `stop_words='english'`
    
    
3. Create a TfidfVectorizer object (let's call it C) with the settings:

    `max_features = 10000, # only top 10k by freq`
    
    `lowercase = False, `
    
    `ngram_range = (1,2), `
    
    `min_df=200,  # note: absolute count of docs`
    
    `max_df=0.95  # note: % of docs` 
    
    
4. Using the training data `documents_train`, along with the ground truth labels `labels_train`, train three Naive Bayes classifiers, corresponding to choices A, B, and C of vectorizer.

5. Normally we'd compute the accuracy of these classifiers on a test set, but for this question we're interested more in the potential upper bound on performance that is achievable with text representation choices A, B, or C.  Thus you should compute, for each of the three classifiers, the accuracy on the *training set*.

6. Your function should return these three accuracy scores as a tuple with three floats: (accuracy_A, accuracy_B, accuracy_C).

It is instructive to examine the difference in accuracy across the three different representations.  

<a href='#toc'>TOC</a>

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [8]:
def train_classifiers(documents_train, labels_train):
    # Create vectorizer A
    A = TfidfVectorizer(max_features=10000,
                        lowercase=False,
                        ngram_range=(1,2),
                        min_df=10,
                        max_df=0.95,
                        stop_words='english')
    
    # Create vectorizer B
    B = CountVectorizer(max_features=10000,
                        lowercase=False,
                        ngram_range=(1,2),
                        min_df=10,
                        max_df=0.95,
                        stop_words='english')
    
    # Create vectorizer C
    C = TfidfVectorizer(max_features=10000,
                        lowercase=False,
                        ngram_range=(1,2),
                        min_df=200,
                        max_df=0.95)
    
    # Transform training data
    X_train_A = A.fit_transform(documents_train)
    X_train_B = B.fit_transform(documents_train)
    X_train_C = C.fit_transform(documents_train)
    
    # Train Naive Bayes classifiers
    clf_A = MultinomialNB().fit(X_train_A, labels_train)
    clf_B = MultinomialNB().fit(X_train_B, labels_train)
    clf_C = MultinomialNB().fit(X_train_C, labels_train)
    
    # Compute accuracy on training set
    accuracy_A = accuracy_score(labels_train, clf_A.predict(X_train_A))
    accuracy_B = accuracy_score(labels_train, clf_B.predict(X_train_B))
    accuracy_C = accuracy_score(labels_train, clf_C.predict(X_train_C))
    
    return (accuracy_A, accuracy_B, accuracy_C)
def answer_text_processing():
    return train_classifiers(documents_train, labels_train)

answer_text_processing()

(0.8352046680222792, 0.7694279904517726, 0.5562726549376713)

In [9]:
# use this cell to explore your solution
# remember to comment the function call before submitting the notebook



In [10]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "1"
print(f"Task {task_id} - AG tests")
stu_ans = answer_text_processing()

print(f"Task {task_id} - your answer: {stu_ans}")

assert isinstance(stu_ans, tuple), "Task 1: Your function should return a tuple."
assert len(stu_ans) == 3, "Task 1: Your tuple should contain three floats."

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Task 1: Your answer at index {i} should be a float number. "

# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 1 - AG tests
Task 1 - your answer: (0.8352046680222792, 0.7694279904517726, 0.5562726549376713)


<a id='Topic2'></a>
## Topic 2 - Latent Semantic Indexing and the vocabulary gap (30 points).

One of the original motivations for Latent Semantic Indexing was overcoming the `vocabulary gap` in information retrieval.
A query like `economic budget` should match strongly against text like `government spending on the economy` even though they don't have any exact keywords in common.

In this question we'll create a demonstration of the power of Latent Semantic Indexing to do semantic matching. In the first part, you'll run LSI and use the reduced document matrix to do semantic matching of a query against other text that has no terms explicitly in common.

In the second part, you'll see how this semantic matching is happening by computing the related terms that are included a query expanded using LSI's latent topics.  

<a href='#toc'>TOC</a>

<a id='t2a'></a>
### Task 2a Use the reduced document matrix from LSI to do semantic matching of a query against a document (15 points).

As a first step, run the code below that we've provided that creates a tf.idf vectorizer and applies it to the 20newsgroups training set. It also runs LSI (in reality a TruncatedSVD) with a latent space of 200 dimensions.

Suppose we have a query "economic budget" that has the tf.idf vector $q$, with shape 1 x num_terms. We can obtain this vector simply by using vectorizer.transform on the text. Think of the matrix $U_k$ as the super operator that converts from original term space to latent semantic space. To expand text $q$ with related terms according to LSI, compute the expanded query $q_k$ using the formula 

$q_k = q \cdot (U_k\Sigma^{-1}_k)$

With this formula, you'll "expand" both the query and the document vectors to add related terms, and then compute the similarity match between them.

Let's walk through these steps (**Note** that in matrix math dimensions and hence order of operations matters!).  
1. The _reduced term matrix_, $U_k$, is _multiplied_ (* operator) with the inverse of matrix $\Sigma_k$, the LSI.singular_values_ matrix. **Note** that $\Sigma_k$ is raised to the power of negative one $\Sigma^{-1}_k$. The reason we invert $\Sigma_k$ is that there is no division operation with matrices so we invert and multiply!
 - Think of this step as forming a normalized scaler for the LSI latent factor weights (the 'topics').  


2. The vectorized query matrix $q$ (or document $d$) is then dotted (dot-product) with the result of $U_k \Sigma^{-1}_k$.

For this question, use cosine similarity to compute the similarity match between any two pieces of text, no matter what their vector representation.

With the formula above, consider the query `"economic budget"` being matched against the (very) short document `"government spending on the economy"`.

Your function should return a tuple of two floats: the cosine similarity score (from sklearn.metrics.pairwise) of (a) the original query and document vectors and (b) the LSI-expanded query and document vectors using the method above.

Did LSI help overcome the vocabulary gap?  

<a href='#toc'>TOC</a>

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.linalg import diagsvd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [12]:
### Run this preamble code to run LSI. We've also given the line of code that gets the resulting U matrix.
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 1), min_df=10, max_df=0.95, stop_words="english", max_features=10000
)  # default English stopwords

tfidf_documents = tfidf_vectorizer.fit_transform(documents_train)

# .get_feature_names() is deprecated in 1.0
# tfidf_feature_names = tfidf_vectorizer.get_feature_names()

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# LSI does truncated SVD on the document-term matrix of tf.idf term-weights.
# The matrix we got back from the vectorizer is a
# document-term matrix, i.e. one row per document.
n_topics = 200
lsi = TruncatedSVD(n_components=n_topics, random_state=0)

# To match the examples and development of LSI in
# our lectures, we're going to
# take the transpose of the document-term matrix to give
# TruncatedSVD the term-document matrix as input.

# This is the matrix U_k:  num_term_features x num_topics
reduced_term_matrix = lsi.fit_transform(np.transpose(tfidf_documents))

In [13]:
def answer_semantic_similarity_a():
    # original vectors
    q = tfidf_vectorizer.transform(["economic budget"])
    d = tfidf_vectorizer.transform(["government spending on the economy"])

    # Expand the query with related terms according to LSI
    Uk = reduced_term_matrix 
    Sk = lsi.singular_values_

    qk = q.dot(Uk.dot(np.diag(1/Sk)))
    dk = (d.dot(Uk)).dot(np.diag(1/Sk))

    # Compute the cosine similarity between the query and document vectors
    similarity_original = cosine_similarity(q, d)
    similarity_lsi = cosine_similarity(qk, dk)

    # Return the cosine similarity scores of the original and LSI-expanded vectors
    result = (similarity_original[0][0], similarity_lsi[0][0])
    # print(result)

    return result
# answer_semantic_similarity_a()

In [14]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "2a"
print(f"Task {task_id} - AG tests")

stu_ans = answer_semantic_similarity_a()
print(f"Task {task_id} - your answer: {stu_ans}")

assert isinstance(stu_ans, tuple), "Task 2a: Your function should return a tuple. "
assert len(stu_ans) == 2, "Task 2a: Your tuple should contain two floats."

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Task 2a: Your answer at index {i} should be a float number. "

# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 2a - AG tests
Task 2a - your answer: (0.0, 0.38756039137547116)


<a id='t2b'></a>
## Task 2b - More understanding of Semantic Matching (15 points).
### Which terms does LSI find similar?

To understand why the LSI-expanded vectors get the results they do, we're going to look at what the operator $U$ does to text. In particular, the term-term matrix $UU^T$ tells us the term expansion behavior of this LSI model. Think of the term-term matrix like an operator that first maps a term to the latent space $L_k$ (using $U$), then back again from $L_k$ to term space (using $U$ transpose). The $(i,j)$ entry of $UU^T$ is a kind of *association weight* between term $i$ and term $j$.

Write a function to get the most related terms (according to LSI) for the word "economy". To do this:

1. Compute the term-term matrix from the matrix U  (the reduced_term_matrix variable).
2. Use the term-term matrix to get the association weights of all words related to the term "economy"
3. Sort by descending weight value.
4. Your function should return the top 5 words and their weights as a list of (string, float) tuples.

Do the related terms match your subjective similarity judgment?  

<a href='#toc'>TOC</a>

In [15]:
def answer_semantic_similarity_b():
    t = "economy"
    top = 5
    term_term_matrix = np.dot(reduced_term_matrix, np.transpose(reduced_term_matrix))
    term_index = tfidf_vectorizer.vocabulary_[t]
    # print("top lsi-related terms for", t, ":")
    top_related_terms_indexes = term_term_matrix[term_index, :].argsort()[::-1]
    # print(top_related_terms_indexes[:5])
    topSet = []
    for i in range(0, top):
        this_term = top_related_terms_indexes[i]
        # print('\t{} ({:.2f})'.format(tfidf_feature_names[this_term],term_term_matrix[term_index, this_term]))
        topSet.append((tfidf_feature_names[this_term], term_term_matrix[term_index, this_term]))
    topSet.sort(key=lambda x: x[1], reverse=True)
    # print(topSet)

    return topSet
# answer_semantic_similarity_b()

In [16]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "2b"
print(f"Task {task_id} - AG tests")

stu_ans = answer_semantic_similarity_b()
print(f"Task {task_id} - your answer:\n{stu_ans}")

assert isinstance(stu_ans, list), "Task 2b: Your function should return a list. "
assert (
    len(stu_ans) == 5
), "Task 2b: Your list should contain five elements (the term, score tuples)."

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, tuple
    ), f"Task 2b: Your answer at index {i} should be a tuple. "
    assert isinstance(
        item[0], str
    ), f"Task 2b: The first element of your tuple at index {i} should be a string. "
    assert isinstance(
        item[1], (float, np.floating)
    ), f"Task 2b: The second element of your tuple at index {i} should be a float. "

# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 2b - AG tests
Task 2b - your answer:
[('government', 0.418059810620016), ('people', 0.25474921519912636), ('money', 0.2071115007881263), ('clinton', 0.1972790419264127), ('tax', 0.17710173696845163)]


<a id='Topic3'></a>
## Topic 3 - Semantic Similarity (20 points total):
**Comparing your ranking with word2vec's ranking.**  
For this task we will be comparing your word similarity determination to that of a pre-trained Word2Vec transformer model, Text8.

### Before proceeding:
 - The following code cell must be executed to load the pre-trained word2vec model.
 - The result is an instance of the class W2VTransformer(size=100, min_count=1, seed=2)
   - `from gensim.sklearn_api import W2VTransformer`
 - For reading the binary file into memory, we now use the Python context handler `with` rather than `open`/`close`. This is considered to be more 'Pythonic.'  

<a href='#toc'>TOC</a>

In [17]:
from gensim.sklearn_api import W2VTransformer

In [18]:
import pickle

with open("../../voc/public/assets/text8_W2V.pickle", "rb") as fh:
    text8_model = pickle.load(fh)

<a id='t3_'></a>
### Task 3 - Ranking words by yourself and then generating word2vec's ranking

The first part of the task is to rearrange the terms in the `my_ranking` tuple based on your subjective impression of how similar they are to the term `"party"`. Encode your impression with terms arranged in **decreasing** order of similarity to the term `"party"` e.g., index 0 holds `"party"`, index 1 the next most similar term all the way to index 8 holding the term you consider the least similar. **This is your personal subjective understanding of how similar these terms are to the word** `"party"`. **Your** arrangement is the correct answer for this task! 

For example, if you believe the term `"event"` is the most similar `"party"`, it should be placed in second position within the tuple and so on. Make sure that your final tuple contain all nine terms, and beginning with `"party"` (index 0).

_ranking tuple original ordering_  
my_ranking = ("party", "bicycle", "vote", "lead", "election", "champagne", "event", "fun", "budget",)  

<a href='#toc'>TOC</a>

In [19]:
# EDIT THE FOLLOWING variable my_ranking.

my_ranking = ("party","lead","vote","fun","event","election","champagne","budget","bicycle",)

In [20]:
# system reference

reference_terms = (
    "party",
    "bicycle",
    "vote",
    "lead",
    "election",
    "champagne",
    "event",
    "fun",
    "budget",
)

<a id='t3a'></a>
### Task 3a - Extract the Text8 model's expression of similarity between "party" and the remaining terms.

#### One way to do this:
1. Use the word2vec `text8_model` to convert `my_ranking` into the corresponding set of word embedding vectors.
2. Determine the cosine similarity of each word embedding relative to the target embedding of `"party"`. Maintain a list of tuples in the format of `(similarity_score, word)`. Do not skip over the embedding for the target word `"party"`, the list should include the similarity to itself.
3. Sort the list in decreasing order. Remember, when sorting elements of type tuple, the first element determines the sorting order and the second element is only considered if there is a tie.
4. Create a list variable called `system_ranking` which contains only the second element of each tuple (the words). 

Your function should return a tuple in the format of `(my_ranking, system_ranking)`.  

<a href='#toc'>TOC</a>

In [21]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.base import TransformerMixin
import numpy as np
from scipy.spatial.distance import cosine

In [22]:
wordvecs = text8_model.transform(my_ranking)   #makes nd.array 
tw_embed = wordvecs[0]
analysis = []
for i in range(0, len(my_ranking)):
    # print("for ",my_ranking[i], wordvecs[i])
    similarity = 1 - cosine(tw_embed, wordvecs[i])
    analysis.append((similarity,my_ranking[i]))
# Sort the list in decreasing order by the first element
analysis.sort(key=lambda x: x[0], reverse=True)

# Extract a list of strings containing only the second element of each tuple
system_ranking = [word for score, word in analysis]
result=(my_ranking, system_ranking)
print(result)

(('party', 'lead', 'vote', 'fun', 'event', 'election', 'champagne', 'budget', 'bicycle'), ['party', 'election', 'vote', 'budget', 'event', 'lead', 'champagne', 'bicycle', 'fun'])


In [23]:
# def rank_words(my_ranking):
#     wordvecs = text8_model.transform(my_ranking)   #makes nd.array 

#     # Determine the cosine similarity of each word embedding relative to the target embedding of "party"
#     similarity_scores = []
#     target_embedding = text8_model['party']
#     for i, word in enumerate(my_ranking):
#         similarity = np.dot(word_embeddings[i], target_embedding) / (np.linalg.norm(word_embeddings[i]) * np.linalg.norm(target_embedding))
#         similarity_scores.append((similarity, word))
    
#     # Sort the list in decreasing order
#     similarity_scores.sort(key=lambda x: x[0], reverse=True)
    
#     # Create a list variable called system_ranking which contains only the second element of each tuple (the words)
#     system_ranking = [x[1] for x in similarity_scores]
    
#     return (my_ranking, system_ranking)
# rank_words(my_ranking)

In [24]:
def answer_word2vec_a():

    wordvecs = text8_model.transform(my_ranking)   #makes nd.array 
    tw_embed = wordvecs[0]
    analysis = []
    for i in range(0, len(my_ranking)):
        # print("for ",my_ranking[i], wordvecs[i])
        similarity = 1 - cosine(tw_embed, wordvecs[i])
        analysis.append((similarity,my_ranking[i]))
    # Sort the list in decreasing order by the first element
    analysis.sort(key=lambda x: x[0], reverse=True)

    # Extract a list of strings containing only the second element of each tuple
    system_ranking = tuple([word for score, word in analysis])
    result=(my_ranking, system_ranking)
    return result
# answer_word2vec_a()

In [25]:
# use this cell to explore your solution
# remember to comment the function call before submitting the notebook

# answer_word2vec_a()

In [26]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "3a"
print(f"Task {task_id} - AG tests")
stu_ans = answer_word2vec_a()

print(f"Task {task_id} - your answer: {stu_ans}")

assert isinstance(stu_ans, tuple), "Task 3a: Your function should return a tuple. "

assert (
    len(stu_ans) == 2
), "Task 3a: Your tuple should contain two elements: a tuple of strings (my_ranking), and a tuple of strings (system_ranking)."

# check my_rankings
assert isinstance(
    stu_ans[0], tuple
), "Task 3a: Your first element must be a tuple (of strings). "

assert len(stu_ans[0]) == len(
    reference_terms
), "Task 3a: Your my_rankings tuple doesn't have the expected number of terms."

assert (
    stu_ans[0][0] == reference_terms[0]
), "Task 3a: Your my_rankings tuple must have 'party' as the first term."

# must be a permutation of the official term set
assert set(stu_ans[0]) == set(
    reference_terms
), "Task 3a: Your my_rankings tuple is not a permutation of the permitted terms."

# check system_rankings
assert isinstance(
    stu_ans[1], tuple
), "Task 3a: Your second element must be a tuple (of strings). "

assert len(stu_ans[1]) == len(
    reference_terms
), "Task 3a: Your system_rankings tuple doesn't have the expected number of terms."

assert (
    stu_ans[0][0] == reference_terms[0]
), "Task 3a: Your system_rankings tuple must have 'party' as the first term."

# must be a permutation of the official term set
assert set(stu_ans[1]) == set(
    reference_terms
), "Task 3a: Your system_rankings tuple is not a permutation of the permitted terms."

# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 3a - AG tests
Task 3a - your answer: (('party', 'lead', 'vote', 'fun', 'event', 'election', 'champagne', 'budget', 'bicycle'), ('party', 'election', 'vote', 'budget', 'event', 'lead', 'champagne', 'bicycle', 'fun'))


<a id='t3b'></a>
### Task 3b Comparing rankings with Spearman correlation coefficient

Given the rankings you generated above in the format of `(my_ranking, system_ranking)`, use the [`scipy.stats.spearmanr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html) function which will return a SciPy object containing the spearman correlation and the p-value of your ranking compared to the system ranking, as a measurement of how well they agree. Convert this spearman result to a tuple, and append that tuple to your previous results, i.e. it should have a format of `(my_ranking, system_ranking, spearman_tuple)`.  

<a href='#toc'>TOC</a>

In [27]:
from scipy import stats

In [28]:
def answer_word2vec_b():
    my_ranking, system_ranking =answer_word2vec_a()
    spearman_result = stats.spearmanr(my_ranking, system_ranking)
    spearman_tuple = tuple(spearman_result)

    results = (my_ranking, system_ranking, spearman_tuple)


    return results
answer_word2vec_b()

(('party',
  'lead',
  'vote',
  'fun',
  'event',
  'election',
  'champagne',
  'budget',
  'bicycle'),
 ('party',
  'election',
  'vote',
  'budget',
  'event',
  'lead',
  'champagne',
  'bicycle',
  'fun'),
 (0.5, 0.17047066078705375))

In [29]:
# use this cell to explore your solution
# remember to comment the function call before submitting the notebook

# answer_word2vec_b()

In [30]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "3b"
print(f"Task {task_id} - AG tests")
stu_ans = answer_word2vec_b()

print(f"Task {task_id} - your answer: {stu_ans}")

assert isinstance(stu_ans, tuple), "Task 3a: Your function should return a tuple. "
assert (
    len(stu_ans) == 3
), "Task 3a: Your tuple should contain three elements: a tuple of strings (my_ranking), a tuple of strings (system_ranking), a tuple with spearman output (2 floats)."

# check my_rankings
assert isinstance(
    stu_ans[0], tuple
), "Task 3a: Your first element must be a tuple (of strings). "
assert len(stu_ans[0]) == len(
    reference_terms
), "Task 3a: Your my_rankings tuple doesn't have the expected number of terms."
assert (
    stu_ans[0][0] == reference_terms[0]
), "Task 3a: Your my_rankings tuple must have 'party' as the first term."
assert set(stu_ans[0]) == set(
    reference_terms
), "Task 3a: Your my_rankings tuple is not a permutation of the permitted terms."  # must be a permutation of the official term set

# check system_rankings
assert isinstance(
    stu_ans[1], tuple
), "Task 3a: Your second element must be a tuple (of strings). "
assert len(stu_ans[1]) == len(
    reference_terms
), "Task 3a: Your system_rankings tuple doesn't have the expected number of terms."
assert (
    stu_ans[0][0] == reference_terms[0]
), "Task 3a: Your system_rankings tuple must have 'party' as the first term."
assert set(stu_ans[1]) == set(
    reference_terms
), "Task 3a: Your system_rankings tuple is not a permutation of the permitted terms."  # must be a permutation of the official term set

# check spearmanr
assert isinstance(
    stu_ans[2], tuple
), "Task 3a: Your third element must be a tuple (of two floats). "
assert (
    len(stu_ans[2]) == 2
), "Task 3a: Your spearman output tuple should contain two floats."
assert isinstance(
    stu_ans[2][0], (float, np.floating)
), "Task 3a: Your spearman corr should be a float. "
assert isinstance(
    stu_ans[2][1], (float, np.floating)
), "Task 3a: Your spearman p-val should be a float. "


# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 3b - AG tests
Task 3b - your answer: (('party', 'lead', 'vote', 'fun', 'event', 'election', 'champagne', 'budget', 'bicycle'), ('party', 'election', 'vote', 'budget', 'event', 'lead', 'champagne', 'bicycle', 'fun'), (0.5, 0.17047066078705375))


<a id='Topic4'></a>
## Topic 4: - Topic Coherence (30 points total).

One measure of topic model quality that is used e.g. to determine the optimal number of topics for a corpus is *topic coherence*. This is a measure of how semantically related the top terms in a topic model are. Topic models with low coherence tend to be filled with seemingly random words and hard to interpret, while high coherence usually indicates a clear semantic theme that's easily understood.

With their ability to represent word semantics, word embeddings are an ideal tool for computing topic coherence. In part 1, you'll implement a simple topic coherence function. In part 2, you'll apply that function to NMF topic modeling to find a setting for the number of topics that gives maximally coherent topic models.

We're going to use the same `text8_model` W2VTransformer object, which implements the word2vec embedding, that you loaded for the previous question.  

<a href='#toc'>TOC</a>

<a id='t4a'></a>
### Task 4a - Average semantic distance as a text coherence measure (15 points).
Implement a function that takes a list of terms (strings) as input and returns a positive float indicating their semantic coherence. Here is the algorithm you should use:

1. For each input term, compute its word2vec embedding vector. One problem you might encounter is that some terms may not exist in the word2vec model. You get a "KeyError" exception when trying to transform that "out-of-vocabulary" term. You should ignore these terms: one way to do this is by wrapping your embedding call with a try/except statement that catches the KeyError and just ignores that word, and continues processing.

2. Once you have the list of embedding vectors for the input terms, compute their pairwise cosine similarity. If there are $n$ embedding vectors, this step will result in an $n x n$ matrix D.  If for some reason there are no input terms remaining (they are all out-of-vocabulary) just return 0.

3. Obviously the most similar word to a term is itself, indicated by a "1" on the diagonal of $D$. But we don't want those: we only care about the pairwise distances to *other* terms, so to deal that case, set the diagonal to zero.

4. Return the mean over all pairwise distances in D (with self-distances set to zero).  This is our simple coherence measure.

Be sure to try it out on some samples. For example, here's what our reference implementation returns:

`topical_coherence(['car', 'airplane', 'taxi', 'bus', 'vehicle', 'transport'])`

0.46063321000999874

`topical_coherence(['apple', 'banana', 'cherry', 'watermelon', 'lemon', 'orange'])`

0.43306025200419956

`topical_coherence(['possible', 'mean', 'volcano', 'feature', 'record', 'quickly'])`

0.1150558124192887

Your function should return the above measure of topic coherence for the following three lists, as a tuple of three corresponding floats:

`['train', 'car', 'bicycle', 'bus', 'vehicle', 'transport']`

`['scsi', 'drive', 'computer', 'storage', 'megabyte']`

`['introduction', 'pickle', 'guard', 'red', 'valiant']`  

<a href='#toc'>TOC</a>

In [31]:
#SAMPLE SETUP
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors

def semantic_coherence(terms):
    # Load pre-trained word2vec model
    model = text8_model.transform(terms) #KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    
    # Compute word2vec embedding vectors for input terms
    embeddings = []
    for term in range(0, len(terms)):
        try:
            embedding = model[term]
            embeddings.append(embedding)
        except KeyError:
            # Ignore out-of-vocabulary terms
            continue
    
    # Handle case where all input terms are out-of-vocabulary
    if len(embeddings) == 0:
        return 0.0
    
    # Compute pairwise cosine similarity
    D = cosine_similarity(embeddings)
    
    # Set diagonal to zero
    np.fill_diagonal(D, 0)
    
    # Compute mean over all pairwise distances
    coherence = np.mean(D)
    
    return coherence
a= ['train', 'car', 'bicycle', 'bus', 'vehicle', 'transport']
print(semantic_coherence(a)) #0.5008174

b =['possible', 'mean', 'volcano', 'feature', 'record', 'quickly']
print(semantic_coherence(b)) #0.115055785

0.5008174
0.115055785


In [32]:


def a4_routine(word_set):
    wordvecs = text8_model.transform(word_set)   #makes nd.array 
    tw_embed = wordvecs[0]
    analysis = []
    for i in range(0, len(word_set)):
        # print("for ",my_ranking[i], wordvecs[i])
        similarity = 1 - cosine(tw_embed, wordvecs[i])
        analysis.append((similarity,word_set[i]))
    # Sort the list in decreasing order by the first element
    analysis.sort(key=lambda x: x[0], reverse=True)

    # Extract a list of strings containing only the second element of each tuple
    system_ranking = tuple([word for score, word in analysis])
    result=(word_set, system_ranking)
    return result
print(a4_routine(a))

(['train', 'car', 'bicycle', 'bus', 'vehicle', 'transport'], ('train', 'bus', 'vehicle', 'car', 'transport', 'bicycle'))


In [33]:
a= ['train', 'car', 'bicycle', 'bus', 'vehicle', 'transport']
b= ['scsi', 'drive', 'computer', 'storage', 'megabyte']
c= ['introduction', 'pickle', 'guard', 'red', 'valiant']
def getScore(terms):
    # Load pre-trained word2vec model
    model = text8_model.transform(terms) #KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    
    # Compute word2vec embedding vectors for input terms
    embeddings = []
    for term in range(0, len(terms)):
        try:
            embedding = model[term]
            embeddings.append(embedding)
        except KeyError:
            # Ignore out-of-vocabulary terms
            continue
    
    # Handle case where all input terms are out-of-vocabulary
    if len(embeddings) == 0:
        return 0.0
    
    # Compute pairwise cosine similarity
    D = cosine_similarity(embeddings)
    
    # Set diagonal to zero
    np.fill_diagonal(D, 0)
    
    # Compute mean over all pairwise distances
    coherence = np.mean(D)
    
    return coherence

def answer_coherence_a():
    result =[]
    total = [a,b,c]
    for i in range(len(total)):
        result.append(getScore(total[i]))

    return tuple(result)
# print(answer_coherence_a())

In [34]:
# use this cell to explore your solution
# remember to comment the function call before submitting the notebook



In [35]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "4a"
print(f"Task {task_id} - AG tests")
stu_ans = answer_coherence_a()

print(f"Task {task_id} - your answer: {stu_ans}")

assert isinstance(stu_ans, tuple), "Task 4a: Your function should return a tuple. "

assert (
    len(stu_ans) == 3
), "Task 4a: Your function should return a tuple of three elements. "

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Task 4a: Your answer at index {i} should be a float number. "

# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 4a - AG tests
Task 4a - your answer: (0.5008174, 0.44538254, 0.10494587)


<a id='t4b'></a>
### Task 4b - Applying semantic coherence to topic model selection (15 points).

Now you'll use the semantic coherence measure you developed in Part 1 with topic models computed using Non-Negative Matrix Factorization.

Implement a simple loop that trains an NMF topic model, for number of topics **from 2 to 10 inclusive**. At each iteration, compute your topic coherence measure on the **top 10** words for each topic. Then compute the *median* topic coherence over all these topic scores.

Your function should return a list of 9 median coherence scores, corresponding to each choice of the number of topics to use with NMF.  Which choice gives the highest median semantic coherence?

When creating the NMF object, use these parameter settings: `random_state=42, init="nndsvd"`.  

<a href='#toc'>TOC</a>

In [36]:
from sklearn.decomposition import NMF

In [37]:
### Use the following code to prepare input to the NMF topic model.
### It assumes you've loaded the 20newgroups variables at the beginning of this assignment
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer_NMF = TfidfVectorizer(
    max_features=20000,
    lowercase=True,
    ngram_range=(1, 1),
    min_df=2,
    max_df=0.05,
    # remove short, non-word-like terms
    token_pattern=r"\b[a-z]{3,12}\b",
    stop_words="english",
)

tfidf_documents_NMF = tfidf_vectorizer_NMF.fit_transform(documents_train)

feature_names_NMF = tfidf_vectorizer_NMF.get_feature_names_out()

In [69]:
def answer_coherence_b():
    topic_index_max = 11
    median_coherence_scores = []
    for topic_index in range(2, topic_index_max):
        nmf = NMF(n_components=topic_index, random_state=42, init="nndsvd")
        W = nmf.fit_transform(tfidf_documents_NMF)  # nmf.fit_transform(tfidf)
        H = nmf.components_
        
        topic_coherence_scores = []
        for topic_idx, topic in enumerate(H):
            top_words_idx = topic.argsort()[:-11:-1]
            top_words = [feature_names_NMF[i] for i in top_words_idx]
            coherence_score = getScore(top_words)
            topic_coherence_scores.append(coherence_score)
        
        median_coherence = np.median(topic_coherence_scores)
        median_coherence_scores.append(median_coherence)  #, topic_index))
    # sorted_list = sorted(median_coherence_scores, key=lambda x: x[0])
    # top_value = sorted_list[0]
    return median_coherence_scores #top_value[1]
# print(answer_coherence_b())

In [70]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
task_id = "4b"
print(f"Task {task_id} - AG tests")
stu_ans = answer_coherence_b()

print(f"Task {task_id} - your answer: {stu_ans}")

assert isinstance(stu_ans, list), "Q4.2: Your function should return a list. "

assert (
    len(stu_ans) == 9
), "Task 4b: Your function should return a list of nine elements (topic count 2 thru 10). "

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Task 4b: Your answer at index {i} should be a float number. "


# Some hidden tests
###
### AUTOGRADER TEST - DO NOT REMOVE
###

del stu_ans

Task 4b - AG tests
Task 4b - your answer: [0.2929746, 0.23651, 0.24788213, 0.25925425, 0.35435927, 0.3755494, 0.3645056, 0.3755494, 0.36107093]


<a href='#toc'>TOC</a>